# Imports

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import collections
from sklearn.preprocessing import MinMaxScaler
from random import randint

import pickle

print(torch.__version__)
import time

import os
import shutil
import copy

!pip install mido
import mido

1.8.1+cu101
     |████████████████████████████████| 61kB 3.5MB/s 


# Load Dataset

In [ ]:
tensorLocation = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Datasets/AllDataSetsCombined/Essential_Midi_ProgressionsOnly/tensor_essentialDataset.pt"
dataset = torch.load(tensorLocation)

print(dataset.shape)

(1416, 4, 3)


# Load Dataset

In [ ]:
tensorLocation = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Datasets/AllDataSetsCombined/Essential_Midi_ProgressionsOnly/tensor_essentialDataset.pt"
dataset = torch.load(tensorLocation)

print(dataset.shape)

(1416, 4, 3)


# Mido Helper Functions

In [ ]:
from mido import Message, MidiFile, MidiTrack
import copy

def listToMidi(chord_list,
               dir_path = './',
               file_name = "new_song.mid",
               program = 12,
               dt = 256,
               veloON = 64,
               veloOFF = 127,
               printOut = True,
               saveMidiFile=True,
               completePath = False):
  """
  converts midi value chords into a midi file with given specifications.
  """
  mid = MidiFile()
  track = MidiTrack()
  mid.tracks.append(track)

  #initialize instrument type
  track.append(Message('program_change', program=12, time=0))
  #Loop through each all chords in list
  for chord in chord_list:
    start = True
    
    #turn chord(s) on
    for note in chord:
      if start == True:
        
        track.append(Message('note_on', note=note, velocity=veloON, time=dt))
        start = False
      else:
        track.append(Message('note_on', note=note, velocity=veloON, time=0))

    start = True
    #turn chord(s) off
    for note in chord:
      if start == True:
        track.append(Message('note_off', note=note, velocity=veloOFF, time=dt))
        start = False
      else:
        track.append(Message('note_off', note=note, velocity=veloOFF, time=0))

  #print created track
  if printOut == True:
    for msg in mid:
      print(msg)
  
  #save midi file
  if saveMidiFile == True and completePath == False:
    complete_path = os.path.join(dir_path,file_name)
    #print(complete_path)
    mid.save(complete_path)
    print("\nfile saved @\n{}".format(complete_path))
  elif completePath != False:
    mid.save(completePath)
##########################################################
##########################################################
#NOT WORKING RIGHT NOW
def midiExtend(file_path,dupNum=2,save_path = False):
  mid = MidiFile(file_path)

  final_tracks = MidiTrack()
  for i in range(dupNum):
    for track in mid:
      #print(track)
      final_tracks.append(track)

  #create new file name
  file_name = "midi_extended"
  num = str(dupNum)
  extension = ".mid"
  full_name = file_name + num + extension

  path = "/".join(file_path.split("/")[0:-1])
  #print("Path {}\n".format(path))
  #print(full_name)
  mid_final = MidiFile()
  mid_final.tracks.append(final_tracks)
  
  final_path = os.path.join(path,full_name)
  mid_final.save(final_path)

## Embedding

In [ ]:
def datasetEmbedder(dataset,inputLength=3):
  """
  dataset: tensorflow dataset with dimensions (total_progs,progression length,notes in chord)
  inputLength: # of chords played.

  Function seperates progressions into "input" & "target"
  """
  assert len(dataset[0]) > inputLength, "inputLength needs to be less than progression length"
  #quadgrams = [(progression[:3],progression[3]) for progression in dataset]
  if inputLength != len(dataset[0]):
    grams = [(progression[:inputLength],progression[inputLength]) for progression in dataset]
  
  return grams

def decoder(code=0):
  """
  code: input encrypted recommendation from model
  """
  string = ix_to_chord[code]
  string = string.replace("["," ")
  string = string.replace("]"," ")
  string = list(string.split(" "))
  final_chord = [int(x) for x in string if x != ""]
  return final_chord

#######################Lookup Table Functions###################################
def createLookup(dataset):
  #create chords as a singular word
  vocab = []
  for prog in dataset:
    for chord in prog:
      tmp_chord = str(chord.numpy()) #convert chord to string
      vocab.append(tmp_chord)
  vocab = set(vocab)
  print("# of Chords in Vocab: {}".format(len(vocab)))

  #create lookup table
  word_to_ix = {word:i for i,word in enumerate(vocab)} #encode
  ix_to_chord = {i:word for i,word in enumerate(vocab)} #decode

  return vocab, word_to_ix, ix_to_chord

def saveLookUp(encode_dict,decode_dict,dirPath):
  """
  encode: word_to_ix
  decode: ix_to_chord
  """
  file_encode = r"lookUp_encode.txt"
  file_decode = r"lookUp_decode.txt"
  #save files
  with open(os.path.join(dirPath,file_encode), "wb") as myFile_encode:
    pickle.dump(encode_dict, myFile_encode)

  with open(os.path.join(dirPath,file_decode), "wb") as myFile_decode:
    pickle.dump(decode_dict, myFile_decode)

  print("Files Saved @:\n{}".format(dirPath))

def loadLookUp(dirPath):
  """
  reverse of saveLookUp()
  """
  file_encode = r"lookUp_encode.txt"
  file_decode = r"lookUp_decode.txt"

  with open(os.path.join(dirPath,file_encode), "rb") as myFile_encode:
    encode_dict = pickle.load(myFile_encode)

  with open(os.path.join(dirPath,file_decode), "rb") as myFile_decode:
    decode_dict = pickle.load(myFile_decode)

  assert len(encode_dict) == len(decode_dict), "encode and decode must be same length"

  vocab = encode_dict #used for the len(vocab) for embed dimensions

  return encode_dict, decode_dict, vocab
########################################################################
########################################################################

In [ ]:
#####################EMBEDDING CREATION####################
CONTEXT_SIZE = 3 #decides how long the input is for next recommendation
EMBEDDING_DIM = 100 #how many dimensions we want inside the linear model to have

###seperates progressions into input and target as a tuple###
quadgrams = datasetEmbedder(dataset,inputLength=CONTEXT_SIZE)

###create new LookUp Table from Dataset###
#vocab, word_to_ix, ix_to_chord = createLookup(dataset) #create new lookup table

#save Lookup Table(s)
saveDir = r"/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/lookup_tables"
#saveLookUp(word_to_ix,ix_to_chord,saveDir)

###load Lookup Table(s)###
word_to_ix, ix_to_chord,vocab = loadLookUp(saveDir)


print("Encoder\n{}".format(word_to_ix))
print("Decoder\n{}".format(ix_to_chord))
print("Number of Chord Options: {}".format(len(word_to_ix)))

Encoder
{'[53 56 60]': 0, '[46 49 53]': 1, '[45 48 52]': 2, '[47 50 53]': 3, '[54 57 61]': 4, '[58 62 65]': 5, '[61 64 67]': 6, '[70 73 76]': 7, '[68 71 74]': 8, '[65 68 71]': 9, '[57 61 64]': 10, '[59 63 66]': 11, '[61 65 68]': 12, '[50 54 57]': 13, '[45 49 52]': 14, '[53 57 60]': 15, '[47 50 54]': 16, '[55 58 62]': 17, '[61 64 68]': 18, '[51 54 57]': 19, '[60 63 67]': 20, '[59 62 66]': 21, '[48 52 55]': 22, '[64 67 70]': 23, '[66 70 73]': 24, '[54 58 61]': 25, '[49 52 55]': 26, '[67 70 74]': 27, '[43 46 50]': 28, '[57 60 64]': 29, '[54 57 60]': 30, '[65 68 72]': 31, '[48 51 55]': 32, '[62 66 69]': 33, '[63 67 70]': 34, '[66 69 73]': 35, '[60 63 66]': 36, '[50 53 56]': 37, '[41 44 48]': 38, '[64 67 71]': 39, '[64 68 71]': 40, '[68 72 75]': 41, '[62 65 69]': 42, '[58 61 65]': 43, '[44 47 51]': 44, '[52 55 59]': 45, '[48 51 54]': 46, '[42 45 49]': 47, '[67 70 73]': 48, '[66 69 72]': 49, '[53 56 59]': 50, '[49 53 56]': 51, '[49 52 56]': 52, '[68 71 75]': 53, '[55 58 61]': 54, '[51 54 58]

In [ ]:
#EXAMPLE: Shows that x list is numbers that pertain to each chord in context
for context, target in quadgrams:
  x = [word_to_ix[str(w.numpy())] for w in context]

print(quadgrams[-1][1].numpy())
print(x)

[58 62 65]
[17, 15, 17]


# Model

In [ ]:
#helping function --> Max Output

def modelRecommend(modelOutput, output_random = False,random_len = 2):
  """
  Used to convert model_output to midi note values
  """
  ##Extract Data from OUTPUT##
  modelOut = modelOutput.cpu()
  modelOut = modelOut.detach().numpy()
  modelOut = list(modelOut)
  finalOut = modelOut
  #print(modelOut.shape)
  #finalOut = list(modelOut[0]) --> ONLY use with "Ngrams_FINAL"
  ############################

  #Method Decision --> Max vs Random
  if output_random == False:
    recommendedChord = finalOut.index(max(finalOut))
  else:
    #create copy with values sorted
    out_sorted = copy.copy(finalOut)
    out_sorted.sort(reverse = True) #Order greatest to least
    out_sorted = out_sorted[:random_len]
    ###############################################################
    #grab indexes of greatest values based on desired input length
    top_choice_indexes = []
    for i in range(len(out_sorted)):
      top_choice_indexes.append(finalOut.index(out_sorted[i]))
    ###############################################################
    #random selector
    recommendedChord = randint(0,len(top_choice_indexes)-1)
  
  return recommendedChord

def modelRecommend_max(modelOutput):
  recommended_max = modelRecommend(modelOutput)
  return recommended_max

def modelRecommend_to_oneHot(vocab_size,recommendedChord,returnNumpy=True):
  """
  Output: 
   1. return numpy array (default)
   2. return torch tensor array (if returnNumpy == False)
  """
  one_hot = np.zeros(vocab_size)
  one_hot[recommendedChord] = 1

  if returnNumpy == True:
    return one_hot
  else:
    tensorArray = torch.from_numpy(one_hot)
    tensorArray = torch.tensor(tensorArray, dtype=torch.long)
    return tensorArray

def testModelOut(modelOutput):
  modelOut = modelOutput.cpu()
  modelOut = modelOut.detach().numpy()
  modelOut = list(modelOut)
  finalOut = modelOut

  out_sorted = copy.copy(finalOut)
  out_sorted.sort(reverse = True) #Order greatest to least

  top_choice_indexes = []
  for i in range(len(out_sorted)):
    top_choice_indexes.append(finalOut.index(out_sorted[i]))

  print(top_choice_indexes)

In [ ]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs
###################################################################################################################
class NGram_LSTM_Model(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size):
    super(NGram_LSTM_Model, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.LSTM = nn.LSTM(embedding_dim, context_size,batch_first=True,num_layers=2)
    self.linear1 = nn.Linear(context_size, vocab_size) #output is length of vocab size because we have 77 options

    self.embedding_dim = embedding_dim
    self.context_size = context_size

  def forward(self, inputs):
    embeds = self.embeddings(inputs).view(1, self.context_size,self.embedding_dim)
    #print(embeds.shape)
    lstm_out, (ht,ct) = self.LSTM(embeds.view(1, self.context_size,self.embedding_dim))
    #print("lstm_out: ",lstm_out)
    out = self.linear1(lstm_out)
    #print(out.shape)
    log_probs = F.log_softmax(out, dim=1)
    #print("log probs: ",log_probs.shape)
    return log_probs

In [ ]:
#initalize GPU & model
gpu = torch.cuda.get_device_name(0)

#intialize
losses = []
loss_function = nn.NLLLoss()
model = NGram_LSTM_Model(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
test = quadgrams[0]
target = test[1]
input = test[0]
print("Input: \n{}\n\nTarget:\n{}\n".format(input,target))

Input_embed = word_to_ix[str(input[-1].numpy())]
Target_embed = word_to_ix[str(target.numpy())]
print("Conversion Test:\nInput Embed: {}\nTarget Embed: {}".format(Input_embed,Target_embed))

Input: 
[[55 59 62]
 [60 64 67]
 [62 66 69]]

Target:
[64 67 71]

Conversion Test:
Input Embed: 33
Target Embed: 39


## Training

In [ ]:
#train
input_data = quadgrams[:]
for epoch in range(200):
    total_loss = 0

    total_accuracy = 0
    count = 0
    correct = 0

    flag = True #change to false for testing
    for context, target in input_data:
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model.forward(context_idxs)
        log_probs = log_probs[0,-1,:] #collect only last time stamp

        #grab recommendation
        recommendedChord = modelRecommend(log_probs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs.view(1,-1), torch.tensor([word_to_ix[str(target.numpy())]], dtype=torch.long).cuda())
        #loss of form (one_hot_model_output,target_index)

        #debug code
        if epoch == 0 and flag == False:
          print("Length of Model Output: ", len(log_probs[0]))
          print("model output: {}\nTarget Chord: {}".format(log_probs,
                                            word_to_ix[str(target.numpy())]))
          flag = True


        #Step 4a. Calculate Accuracy using recommended chord vs expected
        count += 1
        if recommendedChord == word_to_ix[str(target.numpy())]:
          correct += 1

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()

    total_accuracy = correct/count
    print("Epoch: {}  Loss: {}  Accuracy: {}".format(epoch,total_loss,total_accuracy))
    losses.append(total_loss)
    if total_loss < 10:
      #stop training if loss reaches the desired number
      break

#final results of training session
print("Loss: ",losses)

Epoch: 0  Loss: 1458.268099784851  Accuracy: 0.01977401129943503
Epoch: 1  Loss: 1066.5162625908852  Accuracy: 0.02754237288135593
Epoch: 2  Loss: 702.3290989845991  Accuracy: 0.03742937853107345
Epoch: 3  Loss: 388.8028229214251  Accuracy: 0.03531073446327684
Epoch: 4  Loss: 243.07171034999192  Accuracy: 0.04025423728813559
Epoch: 5  Loss: 160.77761341724545  Accuracy: 0.03884180790960452
Epoch: 6  Loss: 109.32313093589619  Accuracy: 0.03248587570621469
Epoch: 7  Loss: 77.80406990158372  Accuracy: 0.03531073446327684
Epoch: 8  Loss: 57.38034307747148  Accuracy: 0.03389830508474576
Epoch: 9  Loss: 43.511497175786644  Accuracy: 0.03177966101694915
Epoch: 10  Loss: 33.641937838168815  Accuracy: 0.03036723163841808
Epoch: 11  Loss: 25.831592370726867  Accuracy: 0.0346045197740113
Epoch: 12  Loss: 19.552832183799183  Accuracy: 0.046610169491525424
Epoch: 13  Loss: 15.406991601608752  Accuracy: 0.03954802259887006
Epoch: 14  Loss: 12.570545945080084  Accuracy: 0.03742937853107345
Epoch: 15 

## Save and Load Weights into Primary model

In [ ]:
#save model weights - MAKE SURE TO NOT OVERWRITE
model_weights_save_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_example.pt"
torch.save(model.state_dict(),model_weights_save_path)
print("weights saved")

weights saved


In [ ]:
#LOAD MODEL WEIGHTS
load_path = "/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/state_dicts/embed_state_dict_model_1inputs_100_dims.pt"
model.load_state_dict(torch.load(load_path))

<All keys matched successfully>

# Evaluation

In [ ]:
#evaluation of model
total_loss = 0

total_accuracy = 0
counter = 0
correct = 0

flag = True #change to false for testing
model.eval()
with torch.no_grad():
  for context, target in quadgrams:
      # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
      # into integer indices and wrap them in tensors)
      context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()

      # Step 3. Run the forward pass, getting log probabilities over next
      # words
      log_probs = model.forward(context_idxs)
      log_probs = log_probs[0,-1,:] #collect only last time stamp

      # Step 4. Compute your loss function. (Again, Torch wants the target
      # word wrapped in a tensor)
      loss = loss_function(log_probs.view(1,-1), torch.tensor([word_to_ix[str(target.numpy())]], dtype=torch.long).cuda())


      #Step 4a. Calculate Accuracy
      recommendedChord = modelRecommend(log_probs)
      counter += 1
      expected = word_to_ix[str(target.numpy())]
      if recommendedChord == expected:
        correct += 1
        #print("Count: {}\nCORRECT\nrecommendedChord: {}\nExpected: {}\n".format(counter,recommendedChord,expected))
      else:
        pass
        print(expected,recommendedChord)
        #print("Count: {}\nFAIL\nrecommendedChord: {}\nExpected: {}\n".format(counter,recommendedChord,expected))

      # Get the Python number from a 1-element Tensor by calling tensor.item()
      total_loss += loss.item()

  total_accuracy = correct/counter
  print("Loss: {}  Accuracy: {}".format(total_loss,total_accuracy))

#Inference Test
print("\n\nInference Speed Test...")
t_start = time.time()
test_data = quadgrams[0]
input = test_data[0]
target = test_data[1]

model.eval()
with torch.no_grad():
  context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in input], dtype=torch.long).cuda()

  log_probs = model(context_idxs)
  log_probs = log_probs[0,0,:] #collect only last time stamp
  ##########################################
  #prints index order based on greatest to least values
  testModelOut(log_probs)
  
  ###########################################
  recommendedChord = modelRecommend(log_probs)
  final_chord = decoder(recommendedChord)
  print("Target Chord: {}, dict_number: {}".format(target,word_to_ix[str(target.numpy())]))
  print("final chord: {} dict_number: {}\n".format(final_chord,recommendedChord))
  print("Eval Time: {}(seconds)".format(time.time() - t_start))

39 34
39 29
75 29
29 11
33 29
39 11
21 63
62 49
39 12
33 49
75 34
33 34
75 11
21 11
29 11
75 29
21 29
62 11
33 11
39 29
75 43
33 11
33 34
29 61
21 61
33 29
39 43
33 34
39 11
33 11
62 11
33 34
75 29
62 29
21 34
33 11
49 61
33 62
39 34
29 34
75 11
33 29
33 29
62 29
75 11
33 11
39 29
29 62
39 29
75 29
33 29
33 61
62 43
62 11
21 11
33 29
63 11
31 11
12 11
34 11
63 11
63 11
34 11
34 11
34 11
63 11
43 11
34 11
43 11
48 11
20 11
34 11
34 11
34 11
12 11
43 11
63 11
20 11
31 11
12 11
63 11
63 11
34 11
43 11
63 11
34 11
31 11
12 11
31 11
31 11
43 11
31 11
34 11
12 11
34 11
63 11
63 11
43 11
43 11
43 11
20 43
31 11
12 11
34 11
12 11
31 11
34 11
12 11
20 11
34 11
63 11
34 11
43 11
31 11
34 11
34 11
20 11
31 11
12 11
20 11
18 11
40 11
24 11
58 11
24 11
53 11
18 11
24 11
53 11
40 20
53 11
53 11
53 20
40 20
40 11
24 63
53 20
58 11
24 11
24 11
18 11
24 20
58 11
40 11
40 34
58 11
18 20
24 11
18 20
53 11
24 34
7 34
24 34
18 11
24 11
18 11
58 11
24 11
58 11
24 11
24 11
40 11
53 11
18 34
24 11
53 11
11 34

# Results
                Eval Accuracy
1. 3 chord inputs: 
2. 2 Chord inputs:  
3. 1 Chord inputs:  

In [ ]:
def combineInputOutput(input_notes_decoded_ndArray,recommendedChord_encoded):
  """
  takes input to model (decoded) form & take recommendedChord (encoded),
  and outputs the two into list form for listToMidi()
  """
  #convert input array to list of lists
  input_chord = input_notes_decoded_ndArray.numpy()
  input_chord = input_chord.toList

  #decode output
  next_chord = decoder(code=recommendedChord_encoded)

  #update original with next_chord
  input_chord.append(next_chord)

  return input_chord
###########################################################################################
###############################SINGLE MODEL RUN FROM NLP_FINAL#############################
def modelRunSingle(dataIn,model,saveMidi=True,output_random=False,save_path=True,count=0):
  #inference for chunk of dataset
  model.eval()
  with torch.no_grad():
    #seperate tuple of input data
    context = dataIn

    midi_out = []
    #time grabber for inference calculations
    dt = time.time()
    #convert input to model form
    context_idxs = torch.tensor([word_to_ix[str(w.numpy())] for w in context], dtype=torch.long).cuda()
    #print(context)

    #run model
    log_probs, = model(context_idxs)
    #print("model output: {}\n".format(log_probs))

    #Chord Selection
    recommendedChord = modelRecommend(log_probs,output_random=output_random)#########################################################################################
    #print("encoded recomendation {}\n".format(recommendedChord))

    #Decode cord
    recommendation = decoder(code=recommendedChord)
    #print("recommendation: {}\n".format(recommendation))

    #append decode to input for musicCompletor
    midi_input = np.append(context,[recommendation],axis=0)
    #print(midi_input)

    #save to midi file
    if save_path == True:
      file_name = BASE_FILE_NAME + str(count) + HANDLE
      file_path = os.path.join(SAVE_DIR,file_name)
    else:
      file_name = BASE_FILE_NAME + str(count) + HANDLE
      file_path = os.path.join(r"./",file_name)
    #print("path\n{}".format(file_path))
    count += 1
    if saveMidi == True:
      listToMidi(midi_input,completePath=file_path,printOut=False)
      print("file saved at:\n{}".format(file_path))

    midi_out.append(midi_input)
    return midi_out

In [ ]:
#variables
SAVE_DIR = r"/content/drive/Shareddrives/Senior Design - Audio Project/MIDI Datasets/Models/NLP Models/midiOutputs"
BASE_FILE_NAME = r"song_"
HANDLE = r".mid"
#input data for testing
test_data = quadgrams[0:4]

i = 0
for context, target in test_data:
  output_midi_list = modelRunSingle(context,model,saveMidi=True,output_random=True,save_path=False,count=i)
  i += 1
  print(output_midi_list)

file saved at:
./song_0.mid
[array([[55, 59, 62],
       [60, 64, 67],
       [62, 66, 69],
       [53, 56, 60]])]
file saved at:
./song_1.mid
[array([[55, 59, 62],
       [54, 57, 60],
       [64, 67, 71],
       [53, 56, 60]])]
file saved at:
./song_2.mid
[array([[55, 59, 62],
       [54, 57, 60],
       [64, 67, 71],
       [53, 56, 60]])]
file saved at:
./song_3.mid
[array([[55, 59, 62],
       [54, 57, 60],
       [59, 62, 66],
       [53, 56, 60]])]


In [ ]:
input_chord = input.numpy()
input_chord = input_chord.tolist()
input_chord

[[55, 59, 62], [60, 64, 67], [62, 66, 69]]

In [ ]:
next_chord = decoder(code=recommendedChord)
next_chord

[60, 63, 67]

In [ ]:
print(input_chord)
print(next_chord)

input_chord.append(next_chord)
listToMidi(input_chord,printOut=False)

[[55, 59, 62], [60, 64, 67], [62, 66, 69], [60, 63, 67]]
[60, 63, 67]
program_change channel=0 program=12 time=0
note_on channel=0 note=55 velocity=64 time=0.26666666666666666
note_on channel=0 note=59 velocity=64 time=0
note_on channel=0 note=62 velocity=64 time=0
note_off channel=0 note=55 velocity=127 time=0.26666666666666666
note_off channel=0 note=59 velocity=127 time=0
note_off channel=0 note=62 velocity=127 time=0
note_on channel=0 note=60 velocity=64 time=0.26666666666666666
note_on channel=0 note=64 velocity=64 time=0
note_on channel=0 note=67 velocity=64 time=0
note_off channel=0 note=60 velocity=127 time=0.26666666666666666
note_off channel=0 note=64 velocity=127 time=0
note_off channel=0 note=67 velocity=127 time=0
note_on channel=0 note=62 velocity=64 time=0.26666666666666666
note_on channel=0 note=66 velocity=64 time=0
note_on channel=0 note=69 velocity=64 time=0
note_off channel=0 note=62 velocity=127 time=0.26666666666666666
note_off channel=0 note=66 velocity=127 time=